### Install Packages

In [ ]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.8 MB/s eta 0:00:00
  Using cached nvidia_cuda

### Import Libraries

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('groq')

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [ ]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./Basics_of_finance.pdf"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [ ]:
len(documents)

86

In [ ]:
documents[4].metadata

{'page_label': '5',
 'file_name': 'Basics_of_finance.pdf',
 'file_path': 'Basics_of_finance.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-06-04',
 'last_modified_date': '2024-06-04'}

### Chunking

In [ ]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [ ]:
len(nodes)

86

In [ ]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'Basics_of_finance.pdf',
 'file_path': 'Basics_of_finance.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-06-04',
 'last_modified_date': '2024-06-04'}

https://chunkviz.up.railway.app/

### Embedding Model

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [ ]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [ ]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [ ]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [ ]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

In [ ]:
query = "Explain market bonds?"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Bonds are debt instruments similar to loans, where the issuer is the debtor and the investor or buyer of the bond is the lender. The issuer promises to pay the interest and the nominal or face value of the security in the future. Bonds are traded securities, which means that the 'lender' does not have to keep the asset until maturity, they can sell it on the secondary market. The interest rate that the bond pays, also called the coupon rate, is not necessarily equal to the return expected by the investors.


https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/